In [ ]:
import os
from google.cloud import documentai_v1 as documentai
from google.cloud import aiplatform
from vertexai.language_models import TextEmbeddingModel

# Set your environment variables for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "preprocessing_credentials.json"

# Document AI: Extract Text from PDF
def extract_text_from_pdf(project_id, location, processor_id, file_path):
    client = documentai.DocumentProcessorServiceClient()

    # Read the file
    with open(file_path, "rb") as file:
        content = file.read()

    # Configure the request
    document = documentai.RawDocument(content=content, mime_type="application/pdf")
    name = f"projects/{project_id}/locations/us/processors/{processor_id}"
    request = {"name": name, "raw_document": document}

    # Process the document
    response = client.process_document(request=request)
    document_text = response.document.text

    return document_text

# Vertex AI: Semantic Chunking with Gemini
def perform_semantic_chunking_with_gemini(project_id, location, text_input):
    # Initialize Vertex AI
    aiplatform.init(project=project_id, location=location)

    # Load Gemini model
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

    # Generate embeddings for semantic chunking
    embeddings = model.get_embeddings([text_input])

    # Dummy implementation for chunking (semantic embeddings clustering)
    # For demonstration: Split text into smaller chunks by semantic similarity.
    # Replace this with advanced clustering logic as per your use case.
    chunk_size = 500  # Characters per chunk
    chunks = [text_input[i:i+chunk_size] for i in range(0, len(text_input), chunk_size)]

    return chunks

# Main Workflow
def main():
    # GCP Configuration
    project_id = "athlyze-446917"
    location = "us-central1"
    processor_id = "a370be5d003f980f"
    file_path = "resistant_research_papers/2102.00836v2.pdf"

    # Step 1: Extract text using Document AI
    print("Extracting text from PDF using Document AI...")
    extracted_text = extract_text_from_pdf(project_id, location, processor_id, file_path)
    print(extracted_text)
    # Step 2: Perform semantic chunking with Gemini
    print("Performing semantic chunking with Vertex AI...")
    semantic_chunks = perform_semantic_chunking_with_gemini(project_id, location, extracted_text)

    # Step 3: Print the semantic chunks
    print("\nSemantic Chunks:")
    for i, chunk in enumerate(semantic_chunks, 1):
        print(f"Chunk {i}:\n{chunk}\n")

if __name__ == "__main__":
    main()


Extracting text from PDF using Document AI...
